# Calcul du State of Health (SoH) pour Mercedes-Benz

Ce notebook explore différentes méthodes pour calculer le State of Health (SoH) des batteries des véhicules Mercedes-Benz en utilisant les données de charge.

## Conclusion

### Méthode 1 : Calcul du SoH avec durée de phase et taux de charge

Nous pouvons calculer le State of Health (SoH) en utilisant la durée de phase calculée (`phase_duration`) combinée avec le `CHARGING_RATE_MEAN` pour estimer l'énergie accumulée pendant les phases de charge.

**Formule utilisée :**
$$ Capacity_{estimated} = \frac{phase\_duration \times CHARGING\_RATE\_MEAN}{\frac{SOC\_DIFF}{100}} $$
$$ SoH = \frac{Capacity_{estimated}}{BATTERY\_NET\_CAPACITY} $$

**Filtres appliqués :**
- `NO_SOC_DATAPOINT > 10` : Filtrer les charges avec peu de points de données
- `SOC_LAST < 99` : Arrêter de prendre en compte les temps après 99% SoC pour réduire la corrélation avec `SOC_LAST`
- `phase_duration < 10` : Filtrer les phases de charge trop longues (en heures)

**Correction de corrélation - Methode 1: régression linéaire :**
Une correction a été appliquée pour réduire la corrélation avec la durée de phase :
$$ SoH_{corrected} = \frac{SoH}{0.0277742 \times phase\_duration + 1.01565} $$

Pas possible de corriger la dépendance au charging rate sans toucher à celle sur la durée de la phase. Donc pour l'instant on reste sur une seule correction de dépendance avec une régression linéaire

**Correction de corrélation - Methode 2 : XGBOOST :**
- POur l'instant pas de conclusion tirée, mais cette méthode pourra être creusé si l'on constate des dépendances importantes


**Agrégation :**
- Agrégation hebdomadaire avec médiane du SoH par VIN et par semaine pour lisser les variations

**Problèmes identifiés :**
- `CHARGING_DURATION_OEM` est beaucoup plus court que la durée réelle de la phase, d'où l'utilisation de `phase_duration` calculée
- Il y a un décalage temporel dans `raw_ts` entre la réception de la valeur `total_charging_duration` et une valeur `soc`
- Il reste des outliers, comprendre pourquoi ils sont présents et comment les supprimer.
- Certaines phase de charges sont très/trop longue.

**Résultat :** En utilisant notre propre durée de phase, nous pouvons calculer plus de valeurs de SoH, mais la variance est élevée et une grande majorité des calculs de SoH sont supérieurs à 100%. Pour le moment, cette méthode nécessite des améliorations supplémentaires.



**Next steps**: 
- outliers:
    - connaitre le nombre d'outliers par vin (modele/vin)
    - Outliers liée au charging rate (proportion)
- phase duration 


Méthode 2: 
$$ Capacity_{estimated} = \frac{TOTAL\_ENERGY\_CHARGED}{\frac{SOC\_DIFF}{100}} $$
- La valeur TOTAL_ENERGY_CHARGED ne semble pas lié directement à l'énergie cumulé pendant une charge, cette valeur n'est pas exploitable en tant que tel



In [ ]:
from core.spark_utils import *
import os
from core.s3.s3_utils import S3Service
import plotly.express as px
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, Window
from core.stats_utils import mask_out_outliers_by_interquartile_range, force_decay
from core.spark_utils import safe_astype_spark_with_error_handling
from core.sql_utils import *
from transform.fleet_info.config import *


## 1. Chargement des données

In [ ]:
spark_session = create_spark_session(os.environ.get('S3_KEY'), os.environ.get('S3_SECRET'))

In [ ]:
s3 = S3Service()

In [ ]:
processed_phase = s3.read_parquet_df_spark(spark_session, f'processed_phases/processed_phases_mercedes_benz.parquet').toPandas()
processed_phase_charging = processed_phase[processed_phase["SOC_DIFF"] > 0].copy()


In [ ]:

raw_tss = s3.read_parquet_df_spark(spark_session, f'raw_ts/mercedes-benz/time_series/raw_ts_spark.parquet')
raw_ts = raw_tss.filter(F.col("date") >= "2025-08-01")
raw_ts = raw_ts.toPandas()

In [ ]:
dtypes_list = { "charging_rate": 'float',
    "energy_charged": 'float',
    "battery_level": 'float',
    "battery_level_at_departure": 'float',
    "displayed_start_state_of_charge": 'float',
    "displayed_state_of_charge": 'float',
    "electric_consumption_rate_since_reset": 'float',
    "electric_consumption_rate_since_start": 'float',
    "estimated_range": 'float',
    "max_range": 'float',
    "odometer": 'float',
    "plugged_in": 'bool',
    "preconditioning_departure_status": 'str',
    "smart_charging_status": 'str',
    "starter_battery_state": 'str',
    "status": 'str',
    "total_charging_duration": 'float',
    "vin": 'str',
}

raw_ts = raw_ts.astype(dtypes_list)

In [ ]:
# Assure-toi que les dates sont au bon format
raw_ts['date'] = pd.to_datetime(raw_ts['date'])
processed_phase_charging['DATETIME_BEGIN'] = pd.to_datetime(processed_phase_charging['DATETIME_BEGIN'])
processed_phase_charging['DATETIME_END'] = pd.to_datetime(processed_phase_charging['DATETIME_END'])

# Tri par VIN et datetime
df_dates = raw_ts.sort_values(['date'])
df_phases = processed_phase_charging.sort_values(['DATETIME_BEGIN'])

merged = pd.merge_asof(
    df_dates,
    df_phases[['DATETIME_BEGIN', 'DATETIME_END', 'VIN', 'PHASE_INDEX', "SOC_DIFF", "SOC_FIRST", "SOC_LAST", "BATTERY_NET_CAPACITY", "CHARGING_DURATION_OEM", "CHARGING_RATE_MEAN", 'ODOMETER_LAST']],
    left_by='vin',
    right_by='VIN',
    left_on='date',
    right_on='DATETIME_BEGIN',
    direction='backward'
)

tss = merged[merged['date'] <= merged['DATETIME_END']]

tss = tss.copy()
tss['PHASE_INDEX'] = tss['PHASE_INDEX'].astype(str)

In [ ]:
processed_phase_charging[['BATTERY_NET_CAPACITY', 'CHARGING_DURATION_OEM', 'CHARGING_RATE_MEAN','ODOMETER_FIRST', 'ODOMETER_LAST', 'RANGE', 'SOC_DIFF', 'SOC_FIRST', 'SOC_LAST',
       'TOTAL_ENERGY_CHARGED']] = processed_phase_charging[['BATTERY_NET_CAPACITY', 'CHARGING_DURATION_OEM', 'CHARGING_RATE_MEAN','ODOMETER_FIRST', 'ODOMETER_LAST', 'RANGE', 'SOC_DIFF', 'SOC_FIRST', 'SOC_LAST',
       'TOTAL_ENERGY_CHARGED']].astype(float)

## 2. Méthode 1 : Calcul du SoH avec durée de phase et taux de charge

Cette première méthode estime l'énergie chargée en utilisant la durée de la phase de charge multipliée par le taux de charge moyen.

### 2.1 Principe de calcul

Nous pouvons estimer l'énergie de charge avec `CHARGING_DURATION_OEM * CHARGING_RATE_MEAN`. En divisant cela par le **SOC_DIFF** pendant la charge, nous obtenons une capacité de batterie estimée.

**Formules :**

$$ Capacity_{estimated} = \frac{CHARGING\_DURATION \times CHARGING\_RATE\_MEAN}{\frac{SOC\_DIFF}{100}} $$

$$ SoH = \frac{Capacity_{estimated}}{BATTERY\_NET\_CAPACITY} $$


### 2.2 Charging duration vs temps de charge calculer pas nous

#### 2.2.1 charging_duration 

**Note :** Nous manquons beaucoup de phases de charge. Nous pouvons utiliser notre propre durée de phase pour calculer plus de valeurs de SoH.


In [ ]:
processed_phase_charging['estimated_capacity'] = (processed_phase_charging['CHARGING_DURATION_OEM'] / 3600) * processed_phase_charging['CHARGING_RATE_MEAN'] / (processed_phase_charging['SOC_DIFF'] / 100)
processed_phase_charging["soh"] = (processed_phase_charging["estimated_capacity"] / processed_phase_charging["BATTERY_NET_CAPACITY"]) 


In [ ]:
px.scatter(processed_phase_charging[(processed_phase_charging['SOC_DIFF'] > 0)], x='ODOMETER_FIRST', y='soh', color='VIN', hover_data=['SOC_DIFF'])

**Note :** Nous manquons beaucoup de phases de charge. Nous pouvons utiliser notre propre durée de phase pour calculer plus de valeurs de SoH.

#### 2.2.2 durée de charge calculée 

In [ ]:
processed_phase_charging['phase_duration'] = (processed_phase_charging["DATETIME_END"] - processed_phase_charging["DATETIME_BEGIN"] ).dt.total_seconds()/3600

In [ ]:
processed_phase_charging['estimated_capacity'] = (processed_phase_charging['phase_duration']) * processed_phase_charging['CHARGING_RATE_MEAN'] / (processed_phase_charging['SOC_DIFF'] / 100)
processed_phase_charging["soh"] = (processed_phase_charging["estimated_capacity"] / processed_phase_charging["BATTERY_NET_CAPACITY"]) 


In [ ]:
px.scatter(processed_phase_charging, x='ODOMETER_FIRST', y='soh', color='VIN', )

### 2.3 Filtrage SoH absurde

#### 2.3.1 Nombre de point par charge

**Observation**: On voit que les charges avec peut de point on des SoH absurdes:
**Cocnlusion**: On filtre les charges qui ont moins de 10 points 

In [ ]:
px.scatter(processed_phase_charging, x='NO_SOC_DATAPOINT', y='soh')

In [ ]:
processed_phase_charging_filtered = processed_phase_charging[processed_phase_charging['NO_SOC_DATAPOINT'] > 10]

#### 2.3.2 Filtrage des valeurs pour le SoC_first, SoC_last, SoC_diff

- SOC_last < 99


In [ ]:
processed_phase_charging_filtered['SOC_MEAN'] = (processed_phase_charging_filtered['SOC_FIRST'] + processed_phase_charging_filtered['SOC_LAST']) / 2

In [ ]:
px.scatter(processed_phase_charging_filtered, x='SOC_FIRST', y='soh', )

In [ ]:
px.scatter(processed_phase_charging_filtered, x='SOC_LAST', y='soh')

**Observation :** Les phases qui commencent ou finissent avec un SoC élevé biaisent les résultats. Il faut arrêter de prendre en compte les temps entre 99% et 100% pour le calcul.

In [ ]:
px.scatter(processed_phase_charging_filtered, x='SOC_DIFF', y='soh')

In [ ]:
processed_phase_charging_soc_filtered = processed_phase_charging_filtered[processed_phase_charging_filtered['SOC_LAST'] < 99].copy()

#### 2.3.3 filtre sur la durée de charge 

- filtre sur la durée des phases < 30h

In [ ]:
px.scatter(processed_phase_charging_soc_filtered, x='phase_duration', y='soh')

In [ ]:
processed_phase_charging_time_filtered = processed_phase_charging_soc_filtered[processed_phase_charging_soc_filtered['phase_duration'] < 10]

#### 2.3.4 Etude sur les outliers restant

- connaitre le nombre d'outliers par vin (modele/vin)
- Outliers liée au charging rate (proportion)

### 2.4 Analyse des dépendances du SoH - METHODE 1 : REGRESSION LINEAIRE


In [ ]:
corr = processed_phase_charging_time_filtered.select_dtypes(include=['float', 'int']).corr()
px.imshow(corr[['soh']].sort_values(by='soh', ascending=False))


**Observation :** correlation avec la durée de la phase

#### 2.4.1 corrélation avec la durée de la phase

In [ ]:
# Create the scatter plot
fig = px.scatter(
    processed_phase_charging_time_filtered[processed_phase_charging_time_filtered['phase_duration'] < 10], 
    x='phase_duration', 
    y='soh', 
    trendline='ols', 
    color='CHARGING_RATE_MEAN'
)

# Display the figure
fig.show()

# Extract and print trendline values
# The trendline is stored as the last trace in the figure
trendline_trace = fig.data[-1]
hovertemplate = trendline_trace.hovertemplate
print("Trendline equation from hovertemplate:")
print(hovertemplate)

In [ ]:
f= lambda x, y: y / (0.0277742 * x + 1.01565)

In [ ]:
processed_phase_charging_time_filtered["soh_corrected"] = processed_phase_charging_time_filtered[["phase_duration", "soh"]].apply(lambda x: f(x['phase_duration'], x['soh']), axis=1)

In [ ]:
px.scatter(processed_phase_charging_time_filtered, x='phase_duration', y='soh_corrected', trendline='ols', color='CHARGING_RATE_MEAN')

In [ ]:
corr = processed_phase_charging_time_filtered.select_dtypes(include=['float', 'int']).corr()
px.imshow(corr[['soh_corrected']].sort_values(by='soh_corrected', ascending=False))


#### 2.4.2 Dépendance au  charging_rate_meean

-> Pas possible de corriger le charging_rate sans avoir d'avoir sur une plus grande corrélation sur la phase duration

In [ ]:
px.scatter(processed_phase_charging_time_filtered, y='soh_corrected', x='CHARGING_RATE_MEAN', trendline='ols')

# Create the scatter plot
fig = px.scatter(
    processed_phase_charging_time_filtered, 
    x='CHARGING_RATE_MEAN', 
    y='soh_corrected', 
    trendline='ols', 
    color='CHARGING_RATE_MEAN'
)

# Display the figure
fig.show()

# Extract and print trendline values
# The trendline is stored as the last trace in the figure
trendline_trace = fig.data[-1]
hovertemplate = trendline_trace.hovertemplate
print("Trendline equation from hovertemplate:")
print(hovertemplate)

In [ ]:
f = lambda x, y: y / (0.00121548 * x + 0.972648)


In [ ]:
processed_phase_charging_time_filtered["CHARGING_RATE_MEAN"].dtype
processed_phase_charging_time_filtered["soh_corrected"].dtype

In [ ]:
processed_phase_charging_time_filtered["soh_corrected_2"] = processed_phase_charging_time_filtered[["CHARGING_RATE_MEAN", "soh_corrected"]].apply(lambda x: f(x['CHARGING_RATE_MEAN'], x['soh_corrected']), axis=1).astype(float)

In [ ]:
px.scatter(processed_phase_charging_time_filtered, x='phase_duration', y='soh_corrected_2', trendline='ols', color='CHARGING_RATE_MEAN')

In [ ]:
corr = processed_phase_charging_time_filtered.select_dtypes(include=['float', 'int']).corr()
px.imshow(corr[['soh_corrected_2']].sort_values(by='soh_corrected_2', ascending=False))


### 2.5. Analyse des dépendances du SoH - MEHODE 2 : XGBOOST

Cette méthode utilise XGBoost (eXtreme Gradient Boosting) pour prédire le SoH en apprenant automatiquement les dépendances complexes entre les différentes features, plutôt que d'appliquer des corrections manuelles.

**Avantages de XGBoost :**
- Apprend automatiquement les dépendances non-linéaires entre les features
- Gère plusieurs features simultanément
- Moins sensible aux outliers que les modèles linéaires
- Fournit l'importance des features pour l'interprétabilité
- Régularisation intégrée pour éviter le surapprentissage


#### 2.5.1 Import de XGBoost et préparation des données


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')


##### 2.5.1.1 Préparation des données pour l'entraînement

Nous utilisons les données filtrées de la Méthode 1 et sélectionnons les features pertinentes pour prédire le SoH.


##### 2.5.2 Ajout de la contrainte : SoH = 1 quand odomètre = 0

Pour chaque VIN, nous ajoutons un point synthétique avec odomètre = 0 et SoH = 1, car un véhicule neuf devrait avoir un SoH de 100%.


In [ ]:
# Ajout de la contrainte : SoH = 1 quand odomètre = 0 pour chaque VIN
# Création de points synthétiques pour chaque VIN unique
print("Ajout de la contrainte : SoH = 1 quand odomètre = 0 pour chaque VIN...")

# S'assurer que VIN est dans le DataFrame
if 'VIN' not in df_xgb_clean.columns:
    df_xgb_clean = df_xgb.reset_index(drop=True)
    df_xgb_clean = pd.concat([df_xgb_clean, df_xgb[['VIN']]], axis=1)
    df_xgb_clean = df_xgb_clean[features + ['soh', 'VIN']].dropna()

unique_vins = df_xgb_clean['VIN'].unique()
synthetic_points = []

for vin in unique_vins:
    # Récupérer les valeurs moyennes/typiques pour ce VIN pour les autres features
    vin_data = df_xgb_clean[df_xgb_clean['VIN'] == vin]
    
    # Créer un point synthétique avec odomètre = 0 et soh = 1
    synthetic_point = {
        'VIN': vin,
        'ODOMETER_FIRST': 0.0,
        'ODOMETER_LAST': 0.0,
        'soh': 1.0,
        # Utiliser des valeurs moyennes pour les autres features
        'phase_duration': vin_data['phase_duration'].median() if len(vin_data) > 0 else 2.0,
        'CHARGING_RATE_MEAN': vin_data['CHARGING_RATE_MEAN'].median() if len(vin_data) > 0 else 7.0,
        'SOC_DIFF': vin_data['SOC_DIFF'].median() if len(vin_data) > 0 else 50.0,
        'SOC_FIRST': 20.0,  # Valeur typique pour une charge
        'SOC_LAST': 70.0,  # Valeur typique pour une charge
        'NO_SOC_DATAPOINT': vin_data['NO_SOC_DATAPOINT'].median() if len(vin_data) > 0 else 20.0,
        'BATTERY_NET_CAPACITY': vin_data['BATTERY_NET_CAPACITY'].iloc[0] if len(vin_data) > 0 else 80.0
    }
    synthetic_points.append(synthetic_point)

# Convertir en DataFrame et ajouter aux données
df_synthetic = pd.DataFrame(synthetic_points)
df_xgb_clean = pd.concat([df_xgb_clean, df_synthetic], ignore_index=True)

print(f"✅ {len(synthetic_points)} points synthétiques ajoutés (odomètre=0, soh=1)")
print(f"Nombre d'observations disponibles (après ajout des contraintes) : {len(df_xgb_clean)}")


In [ ]:
# Préparation des données : utiliser les données filtrées de la Méthode 1
df_xgb = processed_phase_charging_time_filtered.copy()

# Sélection des features pour prédire le SoH
features = [
    'phase_duration',
    'CHARGING_RATE_MEAN',
    'SOC_DIFF',
    'SOC_FIRST',
    'SOC_LAST',
    'ODOMETER_FIRST',
    'ODOMETER_LAST',
    'NO_SOC_DATAPOINT',
    'BATTERY_NET_CAPACITY'
]

# Filtrer les données avec les features disponibles et sans valeurs manquantes
df_xgb_clean = df_xgb[features + ['soh']].dropna()

print(f"Nombre d'observations disponibles : {len(df_xgb_clean)}")
print(f"Nombre de features : {len(features)}")
print(f"\nStatistiques du SoH cible :")
print(df_xgb_clean['soh'].describe())


#### 2.5.3 Séparation train/test et entraînement du modèle


In [ ]:
# Séparation des features et de la variable cible
X = df_xgb_clean[features]
y = df_xgb_clean['soh']

# Split train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Taille du dataset d'entraînement : {len(X_train)}")
print(f"Taille du dataset de test : {len(X_test)}")


In [ ]:
# Création et entraînement du modèle XGBoost
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    objective='reg:squarederror'
)

# Entraînement
print("Entraînement du modèle XGBoost...")
xgb_model.fit(X_train, y_train)
print("✅ Modèle entraîné avec succès!")


#### 2.5.4 Évaluation du modèle


In [ ]:
# Prédictions
y_train_pred = xgb_model.predict(X_train)
y_test_pred = xgb_model.predict(X_test)

# Métriques sur le train
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# Métriques sur le test
mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

print("=" * 50)
print("MÉTRIQUES D'ÉVALUATION")
print("=" * 50)
print(f"\n📊 Dataset d'entraînement :")
print(f"   MAE  : {mae_train:.4f}")
print(f"   RMSE : {rmse_train:.4f}")
print(f"   R²   : {r2_train:.4f}")

print(f"\n📊 Dataset de test :")
print(f"   MAE  : {mae_test:.4f}")
print(f"   RMSE : {rmse_test:.4f}")
print(f"   R²   : {r2_test:.4f}")
print("=" * 50)


#### 2.5.5 Visualisation des prédictions


In [ ]:
# Création d'un DataFrame pour la visualisation
results_df = pd.DataFrame({
    'y_true': y_test.values,
    'y_pred': y_test_pred,
    'error': y_test.values - y_test_pred
})

# Graphique : Prédictions vs Réalité
fig = px.scatter(
    results_df,
    x='y_true',
    y='y_pred',
    title='Prédictions XGBoost vs SoH réel (Test set)',
    labels={'y_true': 'SoH réel', 'y_pred': 'SoH prédit'},
    hover_data=['error']
)

# Ligne de référence (y=x)
min_val = results_df['y_true'].min()
max_val = results_df['y_true'].max()
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='Ligne parfaite (y=x)'
))

fig.show()


In [ ]:
# Graphique : Distribution des erreurs
fig = px.histogram(
    results_df,
    x='error',
    nbins=50,
    title='Distribution des erreurs de prédiction (Test set)',
    labels={'error': 'Erreur (SoH réel - SoH prédit)', 'count': 'Fréquence'}
)
fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Erreur = 0")
fig.show()


#### 2.5.6 Importance des features

XGBoost fournit l'importance de chaque feature, ce qui permet de comprendre quelles variables sont les plus importantes pour prédire le SoH.


In [ ]:
# Récupération de l'importance des features
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Importance des features (du plus au moins important) :")
print("=" * 50)
for idx, row in feature_importance.iterrows():
    print(f"{row['feature']:30s} : {row['importance']:.4f}")
print("=" * 50)

# Visualisation
fig = px.bar(
    feature_importance,
    x='importance',
    y='feature',
    orientation='h',
    title='Importance des features pour la prédiction du SoH',
    labels={'importance': 'Importance', 'feature': 'Feature'}
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


#### 2.5.7 Application du modèle aux données complètes

Nous pouvons maintenant utiliser le modèle entraîné pour prédire le SoH sur toutes les données filtrées.


#### 2.5.8 Application de la contrainte : SoH = 1 quand odomètre = 0

Pour chaque VIN, nous forçons le SoH à 1.0 lorsque l'odomètre est à 0, car un véhicule neuf devrait avoir un SoH de 100%.


In [ ]:
# Application de la contrainte : SoH = 1 quand odomètre = 0
# Pour chaque VIN, si ODOMETER_FIRST = 0, alors soh_xgb = 1
mask_odometer_zero = df_xgb_all['ODOMETER_FIRST'] == 0
n_before = mask_odometer_zero.sum()

if n_before > 0:
    df_xgb_all.loc[mask_odometer_zero, 'soh_xgb'] = 1.0
    print(f"✅ Contrainte appliquée : {n_before} prédictions forcées à SoH=1 (odomètre=0)")
    print(f"\nStatistiques des prédictions XGBoost (après contrainte) :")
    print(df_xgb_all['soh_xgb'].describe())
else:
    print("ℹ️ Aucun point avec odomètre=0 trouvé dans les données")


In [ ]:
# Prédiction sur toutes les données filtrées
df_xgb_all = processed_phase_charging_time_filtered.copy()
df_xgb_all_clean = df_xgb_all[features].dropna()

# Prédictions
soh_xgb_predicted = xgb_model.predict(df_xgb_all_clean)

# Ajout des prédictions au DataFrame
df_xgb_all.loc[df_xgb_all[features].dropna().index, 'soh_xgb'] = soh_xgb_predicted

print(f"Nombre de prédictions effectuées : {len(soh_xgb_predicted)}")
print(f"\nStatistiques des prédictions XGBoost :")
print(df_xgb_all['soh_xgb'].describe())


In [ ]:
# Comparaison visuelle : SoH calculé vs SoH prédit par XGBoost
df_compare = df_xgb_all.dropna(subset=['soh', 'soh_xgb'])
fig = px.scatter(
    df_compare,
    x='soh',
    y='soh_xgb',
    title='Comparaison : SoH calculé (Méthode 1) vs SoH prédit (XGBoost)',
    labels={'soh': 'SoH calculé (Méthode 1)', 'soh_xgb': 'SoH prédit (XGBoost)'},
    color='ODOMETER_FIRST',
    hover_data=['VIN', 'phase_duration', 'CHARGING_RATE_MEAN']
)

# Ligne de référence
min_val = min(df_compare['soh'].min(), df_compare['soh_xgb'].min())
max_val = max(df_compare['soh'].max(), df_compare['soh_xgb'].max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='Ligne parfaite (y=x)'
))

fig.show()


#### 2.5.9 Conclusion de la méthode XGBoost

**Avantages :**
- ✅ Apprend automatiquement les dépendances complexes entre les features
- ✅ Pas besoin de corrections manuelles comme dans la Méthode 1
- ✅ Gère plusieurs features simultanément
- ✅ Fournit l'importance des features pour l'interprétabilité
- ✅ Moins sensible aux outliers que les modèles linéaires

**Points à améliorer :**
- 🔄 Possibilité d'ajuster les hyperparamètres (grid search, random search)
- 🔄 Ajouter plus de features si disponibles (température, historique du véhicule, etc.)
- 🔄 Validation croisée pour une meilleure évaluation
- 🔄 Comparaison avec d'autres algorithmes (Random Forest, LightGBM, etc.)

**Utilisation :**
Le modèle peut être utilisé pour prédire le SoH sur de nouvelles données de charge en utilisant les mêmes features.


In [ ]:
px.scatter(df_xgb_all, x='ODOMETER_LAST', y='soh_xgb', color='VIN')

In [ ]:
# Fréquence de mise à jour : agrégation par semaine
UPDATE_FREQUENCY = pd.Timedelta(days=7)

# Création de la colonne WEEK pour l'agrégation hebdomadaire
df_xgb_all["WEEK"] = (
    pd.to_datetime(df_xgb_all["DATETIME_BEGIN"], format="mixed")
    .dt.floor(UPDATE_FREQUENCY)
    .dt.tz_localize(None)
    .dt.date.astype("datetime64[ns]")
)
# Agrégation hebdomadaire : médiane du SoH par VIN et par semaine
soh_week_median_xgb = df_xgb_all.groupby(["VIN", "WEEK"], as_index=False).agg({
    "soh_xgb": "median",
    "ODOMETER_LAST": "max",
})


soh_var_median_xgb = soh_week_median_xgb.groupby("VIN").agg(
    soh_median = ("soh_xgb", "median"),
    soh_mean = ("soh_xgb", "mean"),
    soh_std = ("soh_xgb", "std"),
    number_week = ("soh_xgb", "count"),
    min = ("soh_xgb", "min"),
    max = ("soh_xgb", "max"),
    
).eval("soh_diff_xgb=max-min")

px.scatter(soh_var_median_xgb, x="number_week", y="soh_diff_xgb", )


In [ ]:
px.scatter(soh_week_median_xgb, x="ODOMETER_LAST", y="soh_xgb", trendline="ols")

## 3. Méthode 2 : Calcul du SoH avec TOTAL_ENERGY_CHARGED

Cette troisième méthode utilise directement la valeur **TOTAL_ENERGY_CHARGED** fournie par Mercedes.

Nous avons la valeur **TOTAL_ENERGY_CHARGED** pendant une charge, nous pouvons l'utiliser avec le **SOC_DIFF** pour estimer la capacité de la batterie.

**Formule :**

$$ Capacity_{estimated} = \frac{TOTAL\_ENERGY\_CHARGED}{\frac{SOC\_DIFF}{100}} $$

In [ ]:
processed_phase_energy = processed_phase_charging.copy()

In [ ]:
def calcul_soh(energie_kwh, delta_soc, capacite_nominale_kwh):
    if delta_soc == 0:
        return np.nan
    estimated_capacity = energie_kwh / (abs(delta_soc) / 100)
    soh = estimated_capacity / capacite_nominale_kwh
    return round(soh, 2) 


processed_phase_energy['soh'] = processed_phase_energy.apply(lambda x: calcul_soh(x['TOTAL_ENERGY_CHARGED'], x['SOC_DIFF'],  x['BATTERY_NET_CAPACITY']) ,axis=1).dropna()


In [ ]:
px.scatter(processed_phase_energy, x='ODOMETER_LAST', y='soh', color='VIN')

#### 4.2 Analyse TOTAL_ENERGY_CHARGED

Le total TOTAL_ENERGY_CHARGED n'est pas logique, il est récupérer a des moment différent de la phase a laquelle il est associé et difficilement associable a la bonne charge. De plus les valeurs ne sont pas logique pour le moment ce qui ne le rend pas utilisable.

In [ ]:
px.scatter(processed_phase_energy[processed_phase_energy['SOC_DIFF'] > 0], x='SOC_DIFF', y='TOTAL_ENERGY_CHARGED', color='VIN')

In [ ]:
px.histogram(processed_phase_energy[['TOTAL_ENERGY_CHARGED']].dropna(), x='TOTAL_ENERGY_CHARGED', nbins=100)

### Conclusion

**Problèmes identifiés :**

- La valeur **TOTAL_ENERGY_CHARGED** n'est pas enregistrée de manière cohérente immédiatement après une session de charge. La période peut ne pas être correcte.
- Les données **TOTAL_ENERGY_CHARGED** sont incohérentes et contiennent des valeurs absurdes.

**Conclusion :** Cette méthode ne peut pas être utilisée pour le moment.

### 5.2 Aggregation à la semaine


In [ ]:
# Fréquence de mise à jour : agrégation par semaine
UPDATE_FREQUENCY = pd.Timedelta(days=7)


In [ ]:
# Création de la colonne WEEK pour l'agrégation hebdomadaire
processed_phase_charging_time_filtered["WEEK"] = (
    pd.to_datetime(processed_phase_charging_time_filtered["DATETIME_BEGIN"], format="mixed")
    .dt.floor(UPDATE_FREQUENCY)
    .dt.tz_localize(None)
    .dt.date.astype("datetime64[ns]")
)

In [ ]:
# Agrégation hebdomadaire : médiane du SoH par VIN et par semaine
soh_week_median = processed_phase_charging_time_filtered.groupby(["VIN", "WEEK"], as_index=False).agg({
    "soh": "median",
    "ODOMETER_LAST": "max",
})



In [ ]:
soh_var_median = soh_week_median.groupby("VIN").agg(
    soh_median = ("soh", "median"),
    soh_mean = ("soh", "mean"),
    soh_std = ("soh", "std"),
    number_week = ("soh", "count"),
    min = ("soh", "min"),
    max = ("soh", "max"),
    
).eval("soh_diff=max-min")

px.scatter(soh_var_median, x="number_week", y="soh_diff", )